In [706]:
import logging

import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb
import seaborn as sns
import talib
from binance.enums import HistoricalKlinesType
from matplotlib import pyplot as plt
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from main_funcs import get_and_update_data

In [707]:
# configs
country=None
joft = "ETHUSDT"
interval = "4h"

# remove small candle 
zero_remover_pct = 0.002

# add history of open_close_pct 
open_close_pct_history_count = 100

# add history of max_low_high_pct 
max_low_high_pct_history_count = -1

# calculate rsi with different lengths 
rsi_ranges = range(14, 23)

# add history of rsi
rsi_history_count = 100

# add moving average of rsi
rsi_ma_lengths = [3,7,9,10,14,20,50,100]
rsi_ma_ranges = [14,19,22]


In [708]:
# validation config

# valid rsi_ma_ranges
for r in rsi_ma_ranges:
    if not r in rsi_ranges:
        raise ValueError("rsi_ma_ranges items is not in rsi_ranges")
    

In [709]:
# get data from database
df_source = get_and_update_data(joft, interval, HistoricalKlinesType.SPOT, country=country)
df_source = df_source[["open", "high", "low", "close", "volume"]]
print(df_source.shape)
df_source.tail(1)

(13877, 5)


,open,high,low,close,volume
open_time,,,,,
2023-12-19 15:30:00,2236.13,2241.27,2179.0,2191.38,85379.1111


In [710]:
# calculate parameters
df_cal = df_source.copy()

# change from open to other columns
df_cal['open_close'] = df_cal.close - df_cal.open
df_cal['open_low'] = df_cal.low - df_cal.open
df_cal['open_high'] = df_cal.high - df_cal.open

# change percent from open to other columns
df_cal['open_close_pct'] = df_cal.open_close / df_cal.open
df_cal['open_low_pct'] = df_cal.open_low / df_cal.open
df_cal['open_high_pct'] = df_cal.open_high / df_cal.open

# price direction
# df_target['open_close_direction'] = np.sign(df_target["open_close"])
# df_target['open_open_high_direction'] = np.sign(df_target["open_low"]).astype(int)
# df_target['open_high_direction'] = np.sign(df_target["open_high"]).astype(int)

# max abs low and high
df_cal['max_low_high_pct'] = np.where(
    df_cal['open_low_pct'].abs() > df_cal['open_high_pct'].abs(),
    df_cal['open_low_pct'],
    df_cal['open_high_pct']
)

# open_close_pct history
for h in range(0, open_close_pct_history_count + 1):
    df_cal[f"open_close_pct__{h}"] = df_cal.open_close_pct.shift(h)

# max_low_high_pct history
for h in range(0, max_low_high_pct_history_count + 1):
    df_cal[f"max_low_high_pct__{h}"] = df_cal.max_low_high_pct.shift(h)

# calculate multi rsi
for rsi in rsi_ranges:
    for h in range(0, rsi_history_count + 1):
        df_cal[f"rsi_{rsi}__{h}"] = talib.RSI(df_cal.close.shift(h), timeperiod=rsi)

# calculate rsi sma 
for ma_length in rsi_ma_lengths:
    for rsi in rsi_ma_ranges:
        df_cal[f"rsi_{rsi}_ma_{ma_length}"] = talib.SMA(df_cal[f"rsi_{rsi}__{0}"], timeperiod=ma_length)
df_cal


C:\Users\sajja\AppData\Local\Temp\ipykernel_93448\531330808.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cal[f"open_close_pct__{h}"] = df_cal.open_close_pct.shift(h)
C:\Users\sajja\AppData\Local\Temp\ipykernel_93448\531330808.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cal[f"open_close_pct__{h}"] = df_cal.open_close_pct.shift(h)
C:\Users\sajja\AppData\Local\Temp\ipykernel_93448\531330808.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

,open,high,low,close,volume,open_close,open_low,open_high,open_close_pct,open_low_pct,...,rsi_22_ma_14,rsi_14_ma_20,rsi_19_ma_20,rsi_22_ma_20,rsi_14_ma_50,rsi_19_ma_50,rsi_22_ma_50,rsi_14_ma_100,rsi_19_ma_100,rsi_22_ma_100
open_time,,,,,,,,,,,,,,,,,,,,,
2017-08-17 07:30:00,301.13,307.96,298.00,307.96,1561.95305,6.83,-3.13,6.83,0.022681,-0.010394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 11:30:00,307.95,312.00,307.00,308.95,1177.71088,1.00,-0.95,4.05,0.003247,-0.003085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 15:30:00,308.95,310.51,303.56,307.06,1882.05267,-1.89,-5.39,1.56,-0.006117,-0.017446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 19:30:00,307.74,312.18,298.21,301.60,1208.05192,-6.14,-9.53,4.44,-0.019952,-0.030968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 23:30:00,301.60,310.85,299.01,302.00,1200.94182,0.40,-2.59,9.25,0.001326,-0.008588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-18 23:30:00,2174.81,2224.00,2167.40,2219.43,59542.14840,44.62,-7.41,49.19,0.020517,-0.003407,...,44.326759,43.652463,44.986429,45.643083,44.433684,46.014287,46.848193,55.873880,56.182436,56.289708
2023-12-19 03:30:00,2219.44,2252.80,2211.75,2237.11,67968.39550,17.67,-7.69,33.36,0.007961,-0.003465,...,44.582334,44.008920,45.192655,45.795664,44.342892,45.851432,46.662658,55.780688,56.097202,56.210049
2023-12-19 07:30:00,2237.11,2253.34,2231.60,2247.99,48095.98420,10.88,-5.51,16.23,0.004863,-0.002463,...,44.982025,44.637508,45.606916,46.131090,44.306914,45.729949,46.513123,55.692188,56.016085,56.134172


In [711]:
# create calculate target
df_target = df_cal.copy()
mabna = 'open_close_pct'

# calculate target
# df_target["target"] = df_target[mabna]
df_target["target"] = np.where(
    df_target[mabna] > zero_remover_pct,
    1,
    np.where(
        df_target[mabna] < -zero_remover_pct,
        2, 0
    )
)


# add log
# df_target["target"] = np.log(df_target["target"])

# shift target the real place
df_target["target"] = df_target["target"].shift(-1)
df_target.value_counts('target')

target
1.0    5896
2.0    5632
0.0    2348
Name: count, dtype: int64

In [712]:
# clean data
df = df_target.copy()
df.dropna(inplace=True)
df.drop(columns=['open', 'high', 'low', 'close', 'volume', 'open_close', 'open_low', 'open_high',
                 'open_close_pct', 'open_low_pct', 'open_high_pct', 'max_low_high_pct'], inplace=True)
df

,open_close_pct__0,open_close_pct__1,open_close_pct__2,open_close_pct__3,open_close_pct__4,open_close_pct__5,open_close_pct__6,open_close_pct__7,open_close_pct__8,open_close_pct__9,...,rsi_14_ma_20,rsi_19_ma_20,rsi_22_ma_20,rsi_14_ma_50,rsi_19_ma_50,rsi_22_ma_50,rsi_14_ma_100,rsi_19_ma_100,rsi_22_ma_100,target
open_time,,,,,,,,,,,,,,,,,,,,,
2017-09-06 15:30:00,0.009857,0.045924,-0.021564,0.039708,-0.020771,0.097183,-0.013963,-0.001956,0.099811,-0.054705,...,39.820389,40.541483,41.135741,55.652748,55.691696,55.619729,57.686315,56.526772,56.008611,0.0
2017-09-06 19:30:00,0.000000,0.009857,0.045924,-0.021564,0.039708,-0.020771,0.097183,-0.013963,-0.001956,0.099811,...,40.777381,41.213459,41.684739,55.247702,55.328298,55.282151,57.746402,56.601846,56.083059,0.0
2017-09-06 23:30:00,0.000000,0.000000,0.009857,0.045924,-0.021564,0.039708,-0.020771,0.097183,-0.013963,-0.001956,...,41.869242,42.008119,42.350334,54.757825,54.892911,54.879014,57.619895,56.524038,56.020959,2.0
2017-09-07 03:30:00,-0.025134,0.000000,0.000000,0.009857,0.045924,-0.021564,0.039708,-0.020771,0.097183,-0.013963,...,42.332705,42.319672,42.589855,54.176388,54.386723,54.412636,57.389798,56.354515,55.873825,2.0
2017-09-07 07:30:00,-0.009247,-0.025134,0.000000,0.000000,0.009857,0.045924,-0.021564,0.039708,-0.020771,0.097183,...,42.765990,42.612756,42.814425,53.514061,53.810183,53.881083,57.133253,56.161638,55.704941,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-18 19:30:00,0.009094,0.006073,-0.012447,-0.002580,-0.009893,-0.017001,0.005761,0.002586,0.003319,-0.003443,...,43.359229,44.832014,45.536318,44.577035,46.218313,47.069844,55.990523,56.285834,56.385285,1.0
2023-12-18 23:30:00,0.020517,0.009094,0.006073,-0.012447,-0.002580,-0.009893,-0.017001,0.005761,0.002586,0.003319,...,43.652463,44.986429,45.643083,44.433684,46.014287,46.848193,55.873880,56.182436,56.289708,1.0
2023-12-19 03:30:00,0.007961,0.020517,0.009094,0.006073,-0.012447,-0.002580,-0.009893,-0.017001,0.005761,0.002586,...,44.008920,45.192655,45.795664,44.342892,45.851432,46.662658,55.780688,56.097202,56.210049,1.0


In [713]:
# create x and y
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

In [714]:
# clean x
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x = mms.fit_transform(x)
x

array([[0.42974245, 0.49689456, 0.3712405 , ..., 0.69962477, 0.66906589,
        0.65473808],
       [0.41138914, 0.42974245, 0.49689456, ..., 0.70113464, 0.67100088,
        0.65669588],
       [0.41138914, 0.41138914, 0.42974245, ..., 0.69795577, 0.66899542,
        0.65506282],
       ...,
       [0.42621243, 0.44958882, 0.4283216 , ..., 0.65174018, 0.65799401,
        0.66003542],
       [0.42044424, 0.42621243, 0.44958882, ..., 0.64951635, 0.65590328,
        0.65804004],
       [0.40157444, 0.42044424, 0.42621243, ..., 0.64381566, 0.65097659,
        0.6534841 ]])

In [715]:
# clean y
y = pd.get_dummies(y)

In [716]:
# create test and train
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1398)


In [717]:
# create model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(11, activation='relu', input_shape=(x_test.shape[1],)),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(y_test.shape[1], activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), batch_size=200)


Epoch 1/100
52/52 [==============================] - 1s 7ms/step - loss: 1.0330 - categorical_accuracy: 0.4195 - val_loss: 1.0366 - val_categorical_accuracy: 0.4257
Epoch 2/100
52/52 [==============================] - 0s 4ms/step - loss: 1.0294 - categorical_accuracy: 0.4195 - val_loss: 1.0351 - val_categorical_accuracy: 0.4257
Epoch 3/100
52/52 [==============================] - 0s 4ms/step - loss: 1.0298 - categorical_accuracy: 0.4206 - val_loss: 1.0393 - val_categorical_accuracy: 0.4001
Epoch 4/100
52/52 [==============================] - 0s 4ms/step - loss: 1.0309 - categorical_accuracy: 0.4087 - val_loss: 1.0363 - val_categorical_accuracy: 0.4257
Epoch 5/100
52/52 [==============================] - 0s 4ms/step - loss: 1.0294 - categorical_accuracy: 0.4157 - val_loss: 1.0345 - val_categorical_accuracy: 0.4257
Epoch 6/100
52/52 [==============================] - 0s 4ms/step - loss: 1.0296 - categorical_accuracy: 0.4233 - val_loss: 1.0375 - val_categorical_accuracy: 0.4260
Epoch 7/10

In [718]:
y_pred = np.argmax(model.predict(x_test), axis=1)
y_test = y_test.values.argmax(axis=1)
print(classification_report(y_true=y_test, y_pred=y_pred))
confusion_matrix(y_test, y_pred)


108/108 [==============================] - 0s 757us/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       599
           1       0.43      0.76      0.55      1464
           2       0.42      0.26      0.32      1376

    accuracy                           0.43      3439
   macro avg       0.28      0.34      0.29      3439
weighted avg       0.35      0.43      0.36      3439


C:\Users\sajja\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sajja\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sajja\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[   0,  448,  151],
       [   0, 1119,  345],
       [   0, 1023,  353]], dtype=int64)